In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
from config import genius_id, genius_secret, genius_token
import time

def parse_info(art, tit):
    art = art.strip()
    if "--" in tit:
        tit = tit[:tit.index("--")]
    tit=tit.strip()
    artist = ""
    title  = ""
    
    for char in art:
        if char == " " or char.isalpha() or char.isnumeric():
            artist += char

    for char in tit:
        if char == " " or char.isalpha() or char.isnumeric():
            title += char
            
    artist = re.sub("[\(\[].*?[\)\]]", "", artist).strip()
    title  = re.sub("[\(\[].*?[\)\]]", "", title).strip()
    return artist, title


base_url  ="https://api.genius.com/search"
params = {
    "q": None
}
headers = {
    "Authorization":"Bearer {}".format(genius_token)
}
harder = []
no_lyr = []
url = "https://genius.com/{}-{}-lyrics"
df = pd.read_csv("data/dataset.csv")
df["Lyrics"] = "None" 
small_data = zip(list(df["Artist"]),list(df["Name"]))
count = 1
for artist,title in small_data:
    a,t = parse_info(artist,title)
    count += 1
    query = a + t
    params["q"] = query
    
    response= requests.get(base_url, params=params,headers=headers)
    if response.status_code != 200:
        time.sleep(10)

        response= requests.get(base_url, params=params,headers=headers)
        if response.status_code != 200:
            harder.append( (artist,title) )
            print( (artist,title) )
            continue
        
    json = response.json()
    results = json.get("response").get("hits")
    if type(results) != list or len(results) == 0:
        harder.append( (artist, title) )
        print( (artist,title) )
        continue

    results = results[0]
    if type(results) != dict:
        harder.append( (artist,title) )
        print( (artist,title) )
        continue
    
    lyric_path = results.get("result").get("path")
    
    if lyric_path == None:
        harder.append( (artist,title) )
        print( (artist,title) )
        continue
        
    lyric_url = "https://genius.com"+lyric_path
    response = requests.get(lyric_url)
    
    if response.status_code != 200:
        harder.append( (artist,title) )
        print( (artist,title) )
        continue
    
    soup = bs(response.text, "html.parser")
    lyrics = soup.find("p")
    
    if lyrics == None:
        continue
    
    lyrics = lyrics.get_text()
    lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics)
    lyrics = lyrics.replace("\n"," ")
    lyrics = lyrics.replace("\t"," ")
    df.loc[(df["Artist"]==artist) & (df["Name"]==title),"Lyrics"] = lyrics

df.to_csv("maybe.csv")

('T-Pain', "Buy U a Drank (Shawty Snappin')")
('Mario Winans', "I Don't Wanna Know (feat. Enya & P. Diddy) - 2016 Remaster")
('Pharrell Williams', "Frontin' (feat. Jay-Z) - Club Mix")
('Lucky Daye', 'Roll Some Mo')
('The-Dream', 'Bedroom')
('J Rice', 'Thank You 4 the Broken Heart 2.0')
('Hi-Five', "She's Playing Hard To Get")
('Ghost Town DJs', "My Boo - Hitman's Club Mix")
('Jeremih', 'Surrounded (feat. Chris Brown & Wiz Khalifa)')
('Mike WiLL Made-It', 'The Mantra (with Pharrell & Kendrick Lamar)')
('Nao', 'Orbit - Live from Air Studios')
('Pink Sweat$', 'Coke & Henny Pt. 1')
('Vedo', 'Semi-Gun')
('Cherrelle', 'Saturday Love - 12" Extended Version')
('Dennis Edwards', "Don't Look Any Further - Single Version")
('James Ingram', 'Yah-Mo Be There')
('Kashif', 'Love Changes - Digitally Remastered 1998')
('Maze', 'Before I Let Go - Edit/Remastered')
('FARR', 'Someway Somehow')
('Intellexual', 'Intlxl')
('Sophie Meiers', 'Bruises and Satin - Demo')
('Savannah Ré', 'Best Is Yet To Come')
('

('Sam Brooks', "Just Thought I'd Let You Know")
('Chandler Stephens', 'Something')
('Ciara Rae', 'Heartbreak Mistake')
('Hayden Coffman', "That's Just Me")
('Co. Townes', 'White Freight Liner')
('Brooke Moriber', 'Steal the Thunder')
('Darrell Rae', 'Proofs in the Bourbon')
('Aaron Mansfield', 'Gone')
('Wild Fire', 'Flaunt')
('Jessie Buckley', 'Glasgow (No Place Like Home) - From “Wild Rose” Soundtrack')
('Rod Melancon', 'The Heartbreakers')
('Sarah Ames', 'Grapevine')
('NeillyRich', 'Hey You')
('The Incognitoz', 'Birds')
('Rob Mayes', 'Closer')
('Zach Neil', 'When She Drinks Whiskey')
('Erica Stone', 'Glass House')
('Waylon Jennings', "Mammas, Don't Let Your Babies Grow Up to Be Cowboys")
('Olivia Newton-John', 'Let Me Be There')
('John Denver', 'Take Me Home, Country Roads - Original Version')
('Eddie Rabbitt', 'Every Which Way but Loose - 2009 Remaster')
('Emmylou Harris', 'Two More Bottles of Wine - Remastered Version')
('Bob Luman', 'Lonely Women Make Good Lovers')
('Ronnie Milsap

('Wolfgang Amadeus Mozart', 'Mozart : Ein musikalischer Spaß K522 : IV Presto')
('Claude Debussy', '2 Arabesques, L. 66: No. 1 in E Major')
('Claudio Monteverdi', 'Ciaccona (Antonio Falconiero ca. 1585-1656)')
('Conlon Nancarrow', 'Studies for Player Piano: Study No. 6')
('Igor Stravinsky', 'Scherzo a la russe (original Jazz band version, 1944)')
('Domenico Scarlatti', 'Sonata in D Major, Kk.430')
('Georg Philipp Telemann', 'Trumpet Concerto in D Major, TWV 51:D7: I. Adagio')
('Gioachino Rossini', 'Guillaume Tell (William Tell): Overture (arr. F. Wrede for 2 pianos 8 hands): Andante -')
('Zoltán Kodály', 'Háry János, Op. 15: II. Viennese Musical Clock')
('Béla Bartók', '3 Hungarian Folksongs from the Csìk District, BB 45b, Sz. 35a')
('RAC', 'Katherine - Piano Edit')
('Sergei Prokofiev', 'Romeo and Juliet Suite No. 3, Op. 101: Aubade: Morning Serenade (Suite No. 3, Op. 101: V)')
('Felix Mendelssohn', 'Songs Without Words, Book VI Opus 67: No. 2 in F-Sharp Minor')
('Muzio Clementi', 'Sym

('Maria Callas', 'Andrea Chenier (1997 Digital Remaster): La mamma morta')
('Franz Schubert', 'Trio pour piano et violoncelle en Mi Bémol Majeur, D. 929, Op. 100: II. Andante con moto')
('John Barry', 'Agnus Dei (based on ADAGIO for Strings)')
('Edward Elgar', 'Cello Concerto in E Minor, Op. 85: III. Adagio')
('Sergei Rachmaninoff', 'Vespers, Op. 37: Nunc dimittis')
('Henry Purcell', 'Dido and Aeneas, Z. 626: Overture')
('Johann Sebastian Bach', 'Die Gitarrenprobe (Arr. After Jesus bleibet meine Freude, BWV 147) - Arr. By Gilvan De Oliveira')
('La Scala', 'Act 3 [Rigoletto} - Lili Marlene')
('Ludwig van Beethoven', 'Piano Trio No.7 in B Flat, Op.97 "Archduke": 3. Andante cantabile, ma però con moto - Poco più adagio')
('Maria Callas', 'La Wally (1997 Digital Remaster): Ebben?...Ne andrò lontana')
('Camille Saint-Saëns', 'Le Carnaval des Animaux, R. 125: 13. Le Cygne')
('Claude Debussy', 'Suite bergamasque, L. 75: No. 3, Clair de lune (Arr. L. Caillet for Orchestra)')
('Giacomo Puccini'

('Joan Tower', 'Violin Concerto')
('Joaquín Turina', 'Danzas fantasticas, Op. 22: Orgia')
('Carl Orff', 'Carmina Burana: O Fortuna')
('Gustav Mahler', 'Symphony No. 2 in C Minor, "Resurrection": I. Allegro maestoso')
('Johannes Brahms', '2つのラプソディ 作品79: Rhapsody In B Minor, Op.79, No.1')
('Vladimir Horowitz', 'Danse macabre, Op. 40 (arr. F. Liszt, V. Horowitz and Y. Sudbin for piano)')
('Ludwig van Beethoven', 'Sonata No. 23 "Appassionata" in F Minor, Op. 57: I. Allegro assai')
('Robert Schumann', 'Fantasie In C, Op.17: 1. Durchaus fantastisch und leidenschaftlich vorzutragen - Im Legendenton - Erstes Tempo')
('Johannes Brahms', 'Symphony No. 4 in E Minor, Op. 98: I. Allegro non troppo')
('Igor Stravinsky', "The Firebird (original version): Scene I: Infernal Dance of Kastchei and his subjects under the Firebird's magic spell")
('Pyotr Ilyich Tchaikovsky', '1812 Festival Overture, Op. 49: 1812 Overture, Op. 49: Finale')
('Franz Schubert', 'Symphony No.9 In C Major, D. 944 - "The Great": 

...

('Orlando Gibbons', 'Fantasia No. 1 for the Great Double Bass')
('Edgar Meyer', '3rd Movement from Amalgamations for Solo Bass - Instrumental')
('Alfred Schnittke', '4 Hymns: II. for cello and double bass')
('Carl Ditters von Dittersdorf', '1. Allegro')
('Igor Stravinsky', 'Pulcinella Suite: VII. Vivo')
('Gavin Bryars', 'Double Bass Concerto "Farewell to St. Petersburg"')
('Aulis Sallinen', 'Elegia Sebastian Knightille (Elegy for Sebastian Knight), Op. 10')
('Gustav Mahler', 'Symphony No. 1 in D Major: III. Feierlich und gemessen, ohne zu schleppen')
('Camille Saint-Saëns', 'Carnival of the Animals: V. The Elephant')
('Johannes Matthias Sperger', 'Double Bass Concerto in D Major: I. Allegro moderato')
('Elliott Carter', 'Figment III')
('Jean Baptist Vanhal', 'Double Bass Concerto in E-Flat Major: I. Allegro moderato')
('Peter Maxwell Davies', 'Strathclyde Concerto No. 7: II. Lento - Allegro')
('Ludwig van Beethoven', 'Symphony No. 5 in C Minor, Op. 67: III. Allegro')
('Giovanni Bottesi

('Tiësto', 'Lose Control')
('Armin van Buuren', 'Lonely For You - Zack Martino Remix')
('Rave Revival Sound System', '#RaveRevival')
('Habstrakt', 'All Night')
('Matt Nash', 'Belong')
('Firebeatz', 'Amsterdam')
('GATTÜSO', 'Say It Right')
('Tiësto', 'Can You Feel It')
('Firebeatz', 'Through My Mind')
('Vigiland', 'Strangers - Tritonal Remix')
('Dillon Francis', 'Lost My Mind')
("Daddy's Groove", 'Been A Long Time')
('Sander van Doorn', '500 (PCM)')
('ATB', 'Heartbeat - Original Mix')
('QUIX', 'Stronger (feat. Elanese)')
('Lookee', 'Mama Say Mama Sa')
('Landis', 'Control')
('Albin Myers', 'Walking On Water')
('SMACK', 'Selecta')
('Ferry Corsten', 'Freefall')
('Anton Powers', 'Make Your Move - Joe Stone Edit')
('Thomas Gold', 'Non-Stop')
('Taryn Manning', 'The Light - Lodato VIP Remix')
('Gryffin', 'Remember (with ZOHARA) - Madison Mars Remix')
('Shift K3Y', 'To The Floor')
('Claude VonStroke', 'Getting Hot')
('Rob & Jack', 'Crank')
("Daddy's Groove", 'Addicted To Drums')
('Kenneth G', '

('Ilan Bluestone', 'Mama Africa')
('East & Atlas', 'Whispers')
('Alexander Koning', 'Winterfelderplatz')
('Stil & Bense', 'Sober')
('Dave Winnel', 'Lily Of The Valley')
('Kolonie', 'Isolated')
('EDX', 'Hip - Original Mix')
('Alex Metric', 'Otic')
('Funkagenda', 'Drift - Original Mix')
('Mark Knight', 'Antidote - Original Club Mix')
('CYA', 'Daybreak - Radio Edit')
('BRKLYN', "Things I've Learned - AUST Remix")
('ARTBAT', "Don't Bring Me Down")
('Johan Gielen', 'Say My Name - Tech Mix')
('Spencer Brown', 'Audio')
('Matan Caspi', 'Eunoia - Original Mix')
('Farius', 'Running Away')
('Fehrplay', 'Rosebud')
('ArtLec', 'Void')
('Energy 52', 'Cafe Del Mar (Tale Of Us Renaissance Remix)')
('Max Freegrant', 'Breathe')
('Tinlicker', 'About You')
('Gai Barone', 'Formal')
('Wally Lopez', 'Space Control')
('Khen', 'Closing Doors - Edit')
('Dezza', 'Expanse')
('Aly & Fila', 'Rebirth - Pablo Artigas Remix')
('Gareth Emery', 'Kingdom United')
('Daniel Wanrooy', 'Cobra')
('Ferry Corsten', "We're Not Go

('Matrix & Futurebound', "Fire - M&F's in Session Edit")
('Andrew Rayel', 'Winterburn - Geert Huinink Orchestral Version')
('Markus Schulz', 'Love Me Like You Never Did - Acoustic Version')
('Morgan Page', 'Open Heart (feat. Lissie) - Bonus Acoustic Mix')
('Patrick Baker', "Don't Think Twice, It's All Right")
('Suyano', 'Overload - Acoustic Version')
('John Dahlbäck', 'Shivers - Acoustic Version')
('John Dahlbäck', 'Walking With Shadows - Acoustic Version')
('Hybrid', "Finished Symphony - Hybrid's Soundtrack")
('Armin van Buuren', 'Embrace - Lowland Classical Remix')
('Grabbitz', '7.6.14 (Intro)')
('BRKLYN', "Can't Get Enough - Unplugged")
('Armin van Buuren', 'Virtual Friend - Acoustic Version')
('Moonbeam', 'Follow Me - Acoustic Version')
('John Dahlbäck', 'Atlantis - Acoustic Version')
('Morgan Page', 'Safe Till Tomorrow (feat. Angelika Vee) - Bonus Acoustic Mix')
('Above & Beyond', 'Tri-State')
('King Arthur', 'Praise You (feat. Michael Meaco) - Chill Mix')
('Skrillex', 'Where Are 

('Moon Vs Sun', 'Lowlight')
('Jenn Grant', 'Raven')
('Luca Fogale', 'Half-Saved')
('Pedro Capó', 'Calma - Remix')
('Marshmello', 'BIBA')
('Mickey Singh', 'Yarri Yeah (feat. Nani Anjali)')
('Shan Vincent De Paul', 'Heaven')
('Harrdy Sandhu', 'Kya Baat Ay')
('Neha Kakkar', 'Aankh Marey (From "Simmba")')
('Zack Knight', 'Bom Diggy Diggy')
('Badshah', 'Akh Lad Jaave (From "Loveyatri")')
('Abeer Arora', 'Drunk on You (feat. Hard Bazy)')
('Guru Randhawa', 'Morni Banke')
('Neha Kakkar', 'Dilbar (From "Satyameva Jayate")')
('RITUAL', 'Love Me Back - Acoustic')
('Sam Smith', 'Dancing With A Stranger (With Normani) - Acoustic')
('LAOISE', 'Again - Acoustic Version')
('George Ezra', 'Hold My Girl (Acoustic Version)')
('George Glew', 'Higher')
('Tim Halperin', 'The Reason')
('Arlissa', 'Let This Go - Acoustic')
('Lady Gaga', "I'll Never Love Again - Film Version - Radio Edit")
('Anne-Marie', 'Perfect To Me')
('Dan Owen', 'Splinter - Acoustic')
('Eves Karydas', 'Further Than The Planes Fly - Acoust

In [5]:
df.tail()

,Unnamed: 0,Genre,Id,Popularity,Is_Exp,Name,Artist,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time_Signature,Lyrics
7238,2706,pop,2iSznonsMCSU95AHXFGVcH,30,False,F**k You,The Sockapellas,0.771,0.421,4,-8.360,1,0.0456,0.27300,0.000000,0.2810,0.773,127.885,4,Look inside Look inside Your tiny mind Now loo...
7239,2707,pop,6ukvsBzq4d1vBsAUmz7ZVt,63,False,Daft Punk,Pentatonix,0.717,0.621,11,-7.588,0,0.0605,0.00403,0.000000,0.1440,0.720,133.014,4,"Buy it, use it, break it, fix it Trash it, cha..."
7240,2708,pop,7zcLCtqtovzOjUIr0GMAPh,24,False,Cruise,Home Free,0.584,0.805,11,-4.057,1,0.1090,0.29800,0.000000,0.1030,0.890,74.938,4,Baby you're a song You make me wanna roll my w...
7241,2709,pop,0bkEVRYVWuKBCWOX6kY2kT,24,False,Creep - Studio Version,Street Corner Symphony,0.546,0.431,7,-8.410,1,0.0305,0.44900,0.000039,0.1000,0.120,93.952,4,None
7242,2710,pop,1sCN83vh7rzQ2d7UGAsoga,24,False,A-Punk,Rockapella,0.795,0.636,2,-5.874,1,0.1900,0.21000,0.000000,0.0622,0.952,87.552,4,None


In [7]:
no_lyr = df[(df["Genre"]=="Classical") & (df["Lyrics"]=="None")]

In [11]:
no_lyr = df[df["Lyrics"]=="None"]

In [16]:
no_lyr["Genre"].value_counts()/df["Genre"].value_counts()

classical    0.904011
country      0.181514
edm_dance    0.530178
hiphop       0.165025
pop          0.156348
rnb          0.088957
rock         0.193634
Name: Genre, dtype: float64

In [14]:
df["Genre"].value_counts()

classical    2792
pop          1599
country       898
edm_dance     845
hiphop        406
rock          377
rnb           326
Name: Genre, dtype: int64